In [ ]:
!pip install --upgrade langchain openai -q
!pip install tiktoken -q
!pip install gradio

In [3]:
import os

# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = "----------------"

In [ ]:
!pip install unstructured -q
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q

In [ ]:
!apt-get install poppler-utils

In [23]:
from langchain.document_loaders import DirectoryLoader

directory = '/content/data'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

1

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

16


In [25]:
print(docs[0].page_content)

The Project Gutenberg eBook of Christmas stories, by Georgene Faulkner

This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook.

Title: Christmas stories

The "Story Lady" series

Author: Georgene Faulkner

Illustrator: Frederic Richardson

Release Date: November 24, 2022 [eBook #69416]

Language: English

Produced by: Charlene Taylor, Chuck Greif and the Online Distributed Proofreading Team at https://www.pgdp.net (This file was produced from images generously made available by The Internet Archive)

*** START OF THE PROJECT GUTENBERG EBOOK CHRISTMAS STORIES ***

CHRISTMAS STORIES

[Illustration]


In [27]:
print(docs[15].page_content)

“How should you like to see my farm?” asked Santa Claus. And the next thing Annie and Willie knew they were in a toyland farm-yard. Houses, fences and barns with stalls for horses and cows, and everything as complete as a real farm. Horses rocked to and fro or rolled about on wheels; toy lambs, so wooly and white, said, “Baa-baa-baa,” when their heads were turned to one side.


In [28]:
import openai
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model_name="text-embedding-ada-002")

query_result = embeddings.embed_query("Hello world")
len(query_result)

1536

In [30]:
from langchain.llms import OpenAI

model_name = "gpt-3.5-turbo"
llm = OpenAI(model_name=model_name)

In [31]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff")  # Modify as needed

def get_answer(query):
  answer = chain.run(input_documents=documents, question=query)
  return answer

query = "What is the document about?"

print(get_answer(query))

The document is about a book of Christmas stories called "Christmas Stories" by Georgene Faulkner.


In [34]:
query = "Who is the author?"
get_answer(query)

'The author is Georgene Faulkner.'

In [35]:
query = "Where are the illustrations from"
get_answer(query)

'The illustrations are from original paintings by Frederic Richardson.'

In [37]:
import gradio as gr
import shutil

# Function to handle the file upload and query
def process_query(file, question):

    # Save the uploaded file to the directory expected by the backend
    file_path = os.path.join(directory, file.name)
    with open(file_path, 'wb') as f:
        shutil.copyfileobj(file, f)

    # Load and split the documents
    documents = load_docs(directory)
    docs = split_docs(documents)

    # Get the answer
    answer = get_answer(question)

    return answer

# Gradio interface
demo = gr.Interface(
    fn=process_query,
    inputs=[
        gr.inputs.File(label="Upload a TXT or PDF file"),
        gr.inputs.Textbox(label="Enter your question")
    ],
    outputs="text",
    title = "AskMeDoc"
)

demo.launch(share=True)

<ipython-input-37-dc78ca3caada>:25: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.File(label="Upload a TXT or PDF file"),
<ipython-input-37-dc78ca3caada>:25: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.File(label="Upload a TXT or PDF file"),
<ipython-input-37-dc78ca3caada>:25: GradioDeprecationWarning: `keep_filename` parameter is deprecated, and it has no effect
  gr.inputs.File(label="Upload a TXT or PDF file"),
<ipython-input-37-dc78ca3caada>:26: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Textbox(label="Enter your question")
<ipython-input-37-dc78ca3caada>:26: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(label="Enter your question"

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7edf54d34e49893feb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
